In [251]:
import numpy as np
import pandas as pd

In [252]:
data = pd.read_csv('HousingData.csv')
display(data.head(15))

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,NaN,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,NaN,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,18.9


Aqui, podemos observar que temos um DataFrame composto por valores numéricos. Essa característica nos permite elaborar uma **Regression Tree**. Vemos que existem certos valores NaN, então vamos trata-los

In [253]:
data.isna().sum()

CRIM       20
ZN         20
INDUS      20
CHAS       20
NOX         0
RM          0
AGE        20
DIS         0
RAD         0
TAX         0
PTRATIO     0
B           0
LSTAT      20
MEDV        0
dtype: int64

Nesse caso remover as linhas que possuem NaN removeriam 79 linhas, o que é uma perda considerável na distribuição de dados

In [254]:
print(len(data))
test = np.unique(data['ZN'])
print (len(test))
print(len(data))
test1 = np.unique(data['INDUS'])
print (len(test1))

506
27
506
77


**Observação:**  
Nas colunas `ZN` e `INDUS` , há muitos dados repetidos. Portanto, a aplicação da **moda** (valor mais frequente) para preencher valores ausentes pode trazer resultados mais confiáveis


In [255]:
data['CHAS'] = data['CHAS'].fillna(0)
data['ZN'] = data['ZN'].fillna(data['ZN'].mode()[0])
data['CRIM'] = data['CRIM'].fillna(data['CRIM'].mean())
data['AGE']  = data['AGE'].fillna(data['AGE'].mean())
data['LSTAT'] = data['LSTAT'].fillna(data['LSTAT'].mean())
data['INDUS'] = data['INDUS'].fillna(data['INDUS'].mode()[0])

In [256]:
data.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [257]:
def entropy(col):
    counts = np.unique(col, return_counts = True)
    size = len(col)
    ent = 0
    for ix in counts[1]:
        pi = ix/size
        ent += -pi*np.log2(pi)
    return ent

In [258]:
def info_gain(df, attr, target):
    valores_unicos = np.sort(np.unique(df[attr]))
    best_split = -np.inf
    best_thresh = 0
    thresholds = [(valores_unicos[i] + valores_unicos[i + 1])/2 for i in range (len(valores_unicos) - 1)]
    for thresh in thresholds:
        left = df[df[attr] <= thresh][target]
        right = df[df[attr] > thresh][target]
        ent_split = len(left)/len(data) * entropy(left) + len(right)/len(data) * entropy(right)
        gain = entropy(df[target]) - ent_split
        if gain > best_split:
            best_split = gain
            best_thresh = thresh
    return best_split, best_thresh

In [259]:
class RegressionTree:
    def __init__(self, depth = 0, max_depth = 6):
        self.attr = None
        self.children = {}
        self.threshold = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
    def train(self, df, features, target):
        if len(np.unique(df[target])) == 1 or len(features) == 1 or self.depth >= self.max_depth:
            self.target = df[target].mean()
            return
        results = [info_gain(df, attr, target) for attr in features]
        gains = [x[0] for x in results]
        best_id = np.argmax(gains)
        best_attr = features[best_id]
        best_gain, best_thresh = results[best_id]
        if best_gain == 0 or np.isnan(best_gain):
            self.target = df[target].mean()
            return

        self.attr = best_attr
        self.threshold = best_thresh
        self.children = {}
        
        left = df[df[best_attr] <= best_thresh]
        right = df[df[best_attr] > best_thresh]
        self.children['left'] = RegressionTree(self.depth + 1, max_depth=self.max_depth)
        self.children['left'].train(left, features, target)
        self.children['right'] = RegressionTree(self.depth + 1, max_depth = self.max_depth)
        self.children['right'].train(right, features, target)
    def predict (self,row):
        if self.attr is None or self.children == {}:
            return self.target
        val = row[self.attr]
        if self.threshold is not None:
            if val <= self.threshold:
                return self.children['left'].predict(row)
            else:
                return self.children['right'].predict(row)
                
        

In [260]:
tree = RegressionTree(max_depth=6)
x = data.drop(columns = ['MEDV'])
x_list = list(x.columns)
tree.train(data, x_list , 'MEDV')

In [261]:
for i in range(len(data)):
    test_row = data.iloc[i]
    print(f"Exemplo {i+1}: Previsto={tree.predict(test_row)}, Real={test_row['MEDV']}")

Exemplo 1: Previsto=25.009523809523817, Real=24.0
Exemplo 2: Previsto=22.073333333333334, Real=21.6
Exemplo 3: Previsto=29.2375, Real=34.7
Exemplo 4: Previsto=27.85, Real=33.4
Exemplo 5: Previsto=28.200000000000003, Real=36.2
Exemplo 6: Previsto=25.009523809523817, Real=28.7
Exemplo 7: Previsto=21.823809523809523, Real=22.9
Exemplo 8: Previsto=18.6, Real=27.1
Exemplo 9: Previsto=11.760000000000002, Real=16.5
Exemplo 10: Previsto=18.525, Real=18.9
Exemplo 11: Previsto=18.6, Real=15.0
Exemplo 12: Previsto=21.823809523809523, Real=18.9
Exemplo 13: Previsto=21.95, Real=21.7
Exemplo 14: Previsto=30.39090909090909, Real=20.4
Exemplo 15: Previsto=20.408333333333335, Real=18.2
Exemplo 16: Previsto=30.39090909090909, Real=19.9
Exemplo 17: Previsto=30.39090909090909, Real=23.1
Exemplo 18: Previsto=20.116666666666667, Real=17.5
Exemplo 19: Previsto=20.408333333333335, Real=20.2
Exemplo 20: Previsto=20.408333333333335, Real=18.2
Exemplo 21: Previsto=11.760000000000002, Real=13.6
Exemplo 22: Previs

In [262]:
def mse(y_test, y_pred):
    return np.mean((y_test - y_pred) ** 2)
y_test = data['MEDV'].values
y_pred = [tree.predict(data.iloc[i]) for i in range (len(data))]
val = mse(y_test, y_pred)
rmse = np.sqrt(val)

def rform(y_test, y_pred):
    result = 1 - np.sum((y_test - y_pred)**2)/ np.sum(((y_test - y_test.mean())**2))
    return result
val_r = rform(y_test, y_pred)

                                                    
print(f"R-Squared: {val_r}")
print(f"MSE: {val}")
print(f"RMSE:{rmse}")


R-Squared: 0.8119628967774553
MSE: 15.874008794938316
RMSE:3.984219973211609


## Avaliação de Overfitting do Modelo

**Profundidade 2: MSE = 32.52   | RMSE = 5.70**

**Profundidade 3: MSE = 24 ,77  | RMSE = 4.977**

**Profundidade 4: MSE = 19.73   | RMSE = 4.42**

**Profundidade 5: MSE = 18.1733 | RMSE: 4.26**

**Profundidade 6: MSE = 15.874  | RMSE: 3.98**

**profundidade 7: MSE = 10.12   | RMSE: 3.18**

Observa-se que, para a profundidade 2, o MSE é significativamente elevado, indicando que o modelo possui baixa capacidade de ajuste. Apesar de generalizar bem, a confiabilidade das previsões é limitada.

O objetivo neste contexto é encontrar uma profundidade que ofereça equilíbrio entre **generalização** e **precisão**. As profundidades 4 e 5 se destacam por apresentarem valores de MSE e RMSE mais baixos, além de não exibirem variações bruscas em relação às demais configurações, sugerindo uma boa relação entre ajuste aos dados e capacidade de generalização.

No entanto, para garantir maior confiabilidade na escolha do modelo, é fundamental analisar conjuntamente os valores de MSE e RMSE tanto para os dados de teste quanto para os dados de treino, assegurando que não haja discrepâncias significativas entre eles.

In [263]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['MEDV'])
y = data['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_data = X_train.copy()
train_data['MEDV'] = y_train.values

tree = RegressionTree(max_depth=6)
tree.train(train_data, list(X_train.columns), 'MEDV')
y_train_pred = [tree.predict(train_data.iloc[i]) for i in range(len(train_data))]
test_data = X_test.copy()
test_data['MEDV'] = y_test.values
y_test_pred = [tree.predict(test_data.iloc[i]) for i in range(len(test_data))]

mse_train = mse(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = rform(y_train, y_train_pred)

mse_test = mse(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
r2_test = rform(y_test, y_test_pred)

print(f"Treino - MSE: {mse_train:.2f}, RMSE: {rmse_train:.2f}, R²: {r2_train:.2f}")
print(f"Teste  - MSE: {mse_test:.2f}, RMSE: {rmse_test:.2f}, R²: {r2_test:.2f}")

Treino - MSE: 17.04, RMSE: 4.13, R²: 0.80
Teste  - MSE: 20.20, RMSE: 4.49, R²: 0.72


Profundidade 6
-
**Treino - MSE: 17.04, RMSE: 4.13, R²: 0.80**

**Teste  - MSE: 20.20, RMSE: 4.49, R²: 0.72**

Profundidade 5
-
**Treino - MSE: 24.26, RMSE: 4.93, R²: 0.72**

**Teste  - MSE: 20.15, RMSE: 4.49, R²: 0.73**


Profundidade 4
-
**Treino - MSE: 26.49, RMSE: 5.15, R²: 0.70**

**Teste  - MSE: 21.99, RMSE: 4.69, R²: 0.70**

Observa-se que as profundidades 4, 5 e 6 apresentam pouca variação entre os valores de MSE, RMSE e R², tanto nos dados de treino quanto nos de teste. Como buscamos maior precisão sem comprometer a capacidade de generalização do modelo, optamos pela profundidade 6. Essa configuração proporciona o melhor equilíbrio entre desempenho e generalização, garantindo a maior precisão dentre as opções analisadas, sem incorrer em overfitting.
